In [7]:
import os
import sqlite3
import logging
from datetime import datetime
from dotenv import load_dotenv
from flask import Flask, request, jsonify
from twilio.twiml.messaging_response import MessagingResponse
from twilio.rest import Client
from flask_mpesa import MpesaAPI

# Load environment variables
load_dotenv()

# Initialize Flask app
app = Flask(__name__)

# Twilio setup
twilio_client = Client(os.getenv('TWILIO_ACCOUNT_SID'), os.getenv('TWILIO_AUTH_TOKEN'))
TWILIO_WHATSAPP = os.getenv('TWILIO_WHATSAPP_NUMBER')
ADMIN_PHONE = os.getenv('ADMIN_PHONE')

# M-Pesa setup
mpesa_api = MpesaAPI(app)
app.config["API_ENVIRONMENT"] = os.getenv('MPESA_ENVIRONMENT')
app.config["APP_KEY"] = os.getenv('MPESA_CONSUMER_KEY')
app.config["APP_SECRET"] = os.getenv('MPESA_CONSUMER_SECRET')
SHORTCODE = os.getenv('MPESA_SHORTCODE')

# Database setup
conn = sqlite3.connect('contributions.db', check_same_thread=False)
cursor = conn.cursor()
cursor.execute('''CREATE TABLE IF NOT EXISTS contributions
                  (phone TEXT PRIMARY KEY, amount REAL DEFAULT 0, name TEXT)''')
conn.commit()

# Set up logging
logging.basicConfig(filename='whatsapp_bot.log', level=logging.INFO, 
                    format='%(asctime)s - %(levelname)s - %(message)s')

# Database helper functions
def update_contribution(phone, amount, name=None):
    cursor.execute("SELECT amount FROM contributions WHERE phone=?", (phone,))
    row = cursor.fetchone()
    if row:
        new_amount = row[0] + amount
        cursor.execute("UPDATE contributions SET amount=? WHERE phone=?", (new_amount, phone))
    else:
        cursor.execute("INSERT INTO contributions (phone, amount, name) VALUES (?, ?, ?)", 
                       (phone, amount, name or phone))
    conn.commit()

def get_user_contribution(phone):
    cursor.execute("SELECT amount FROM contributions WHERE phone=?", (phone,))
    row = cursor.fetchone()
    return row[0] if row else 0

def get_total():
    cursor.execute("SELECT SUM(amount) FROM contributions")
    return cursor.fetchone()[0] or 0

def get_list():
    cursor.execute("SELECT phone, amount FROM contributions")
    return "\n".join([f"{row[0]}: KES {row[1]}" for row in cursor.fetchall()])

def send_whatsapp(to, body):
    twilio_client.messages.create(from_=TWILIO_WHATSAPP, body=body, to=f'whatsapp:{to}')

# WhatsApp Webhook
@app.route('/whatsapp', methods=['POST'])
def whatsapp_webhook():
    from_number = request.values.get('From').replace('whatsapp:', '')
    body = request.values.get('Body', '').lower().strip()
    
    resp = MessagingResponse()
    
    # Log the incoming message
    logging.info(f"Received from {from_number}: {body}")
    
    if body in ['help', 'dashboard']:
        msg = ("Welcome to Contribution Bot!\nOptions:\n"
               "- 'my contribution', 'view contribution', 'check contribution', or 'my total' to see your total\n"
               "- 'total' for group total\n"
               "- 'list' (admin only)\n"
               "- 'deposit <phone> <amount>' (admin only)\n"
               "- Message admin: Type your query.")
        resp.message(msg)
    elif body in ['my contribution', 'view contribution', 'check contribution', 'contribution', 'my total']:
        amount = get_user_contribution(from_number)
        if amount > 0:
            resp.message(f"Your contribution: KES {amount:.2f}\nCommand processed successfully.")
        else:
            resp.message(f"You haven't contributed yet. Send a payment to Paybill {SHORTCODE} or contact the admin.")
        logging.info(f"Contribution check for {from_number}: KES {amount:.2f}")
    elif body == 'total':
        total = get_total()
        resp.message(f"Total group contributions: KES {total:.2f}")
        logging.info(f"Total requested by {from_number}: KES {total:.2f}")
    elif body == 'list':
        if from_number == ADMIN_PHONE:
            list_str = get_list()
            resp.message(f"Contributors:\n{list_str}")
            logging.info(f"List requested by admin {from_number}")
        else:
            resp.message("This command is for admins only.")
            logging.warning(f"Non-admin {from_number} attempted 'list' command")
    elif body.startswith('deposit'):
        if from_number == ADMIN_PHONE:
            parts = body.split()
            if len(parts) == 3:
                try:
                    target_phone, amount = parts[1], float(parts[2])
                    update_contribution(target_phone, amount)
                    resp.message(f"Deposited KES {amount:.2f} for {target_phone}")
                    send_whatsapp(target_phone, f"Admin added KES {amount:.2f} to your contribution.")
                    logging.info(f"Admin {from_number} deposited KES {amount:.2f} for {target_phone}")
                except ValueError:
                    resp.message("Invalid amount. Usage: deposit <phone> <amount>")
                    logging.error(f"Invalid deposit by {from_number}: {body}")
            else:
                resp.message("Usage: deposit <phone> <amount>")
                logging.error(f"Invalid deposit syntax by {from_number}: {body}")
        else:
            resp.message("This command is for admins only.")
            logging.warning(f"Non-admin {from_number} attempted 'deposit' command")
    else:
        # Forward to admin as query
        send_whatsapp(ADMIN_PHONE, f"Query from {from_number}: {body}")
        resp.message("Your message has been sent to the admin. They'll reply soon.")
        logging.info(f"Query forwarded from {from_number} to admin")
    
    return str(resp)

# M-Pesa Register
@app.route('/register_mpesa', methods=['GET'])
def register_mpesa():
    public_url = os.getenv('PUBLIC_URL', 'https://your-ngrok-url.com')  # Update with ngrok or production URL
    reg_data = {
        "shortcode": SHORTCODE,
        "response_type": "Completed",
        "confirmation_url": f"{public_url}/mpesa/confirmation",
        "validation_url": f"{public_url}/mpesa/validation"
    }
    response = mpesa_api.C2B.register(**reg_data)
    return jsonify(response)

# M-Pesa Validation
@app.route('/mpesa/validation', methods=['POST'])
def mpesa_validation():
    data = request.get_json()
    logging.info(f"M-Pesa validation: {data}")
    return jsonify({"ResultCode": 0, "ResultDesc": "Accepted"})

# M-Pesa Confirmation
@app.route('/mpesa/confirmation', methods=['POST'])
def mpesa_confirmation():
    data = request.get_json()
    logging.info(f"M-Pesa confirmation: {data}")
    if data['TransactionType'] == 'Pay Bill':
        phone = data['MSISDN']
        amount = float(data['TransAmount'])
        update_contribution(phone, amount)
        send_whatsapp(phone, f"Contribution received: KES {amount:.2f}. Total now: KES {get_user_contribution(phone):.2f}")
        send_whatsapp(ADMIN_PHONE, f"New contribution from {phone}: KES {amount:.2f}. Group total: KES {get_total():.2f}")
    return jsonify({"ResultCode": 0, "ResultDesc": "Accepted"})

if __name__ == '_main_':
    app.run(port=5000, debug=True)